In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings
import seaborn as sns

warnings.filterwarnings('ignore')

# 교통인프라 데이터 불러오기

In [19]:
data=pd.read_csv('/content/인프라지수_자료모음.csv')

In [20]:
data3=data.copy()

# skew가 3이상인 변수: 정류장밀도, 철도역밀도, 1인당 대중교통 이용량-> 로그 변환 진행

In [21]:
#로그변환 (왜도가 3이상인 경우에 로그변환진행함)
data3['정류장 밀도'] = np.log1p(data3['정류장 밀도'])
data3['철도역 밀도'] = np.log1p(data3['철도역 밀도'])
data3['1인당 대중교통 이용량'] = np.log1p(data3['1인당 대중교통 이용량'])

# 정규화 진행

In [22]:
from sklearn.preprocessing import StandardScaler
X = data3.drop(['시군구코드', '발생량'], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 정규화된 값으로 원래 컬럼 덮어쓰기
data3[X.columns] = X_scaled

# 회귀분석 진행
x: 시군구코드와 발생량을 제외한 변수들
y: 발생량

In [23]:
import pandas as pd
from sklearn.linear_model import LinearRegression


# X와 y 분리
X = data3.drop(['시군구코드','발생량'],axis=1)
y = data3['발생량']

# 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X, y)

# 회귀 계수 및 절편 출력
coefficients = pd.Series(model.coef_, index=X.columns)
intercept = model.intercept_

print("회귀 계수:")
print(coefficients)
print("\n절편 (intercept):", intercept)

# R^2 점수 확인 (모델 설명력)
print("\n설명력 (R^2):", model.score(X, y))

회귀 계수:
평균 접근 시간(분)     -1.255697e+06
정류장 밀도          -3.551990e+04
철도역 밀도          -4.605759e+05
1인당 대중교통 이용량     1.883385e+06
평균 환승 횟수        -4.009734e+06
평균 환승 대기시간       9.717854e+03
대중교통 환승 이용 비율    5.219492e+06
환승 발생량           4.941430e+07
평균 통행거리(km)     -3.865350e+05
접근 가능 시설 수(개)    3.314095e+06
dtype: float64

절편 (intercept): 35340341.299559474

설명력 (R^2): 0.9522670569872287


회귀계수를 모두 더한 뒤 각 계수를 정규화(총합으로 나눔)하여 가중치로 활용,
해당 가중치를 변수값에 곱해 만든 "교통인프라수준 지수"를 만듦

In [24]:
norm_weights = model.coef_ / np.sum(np.abs(model.coef_))  # 절댓값 기준 합 1
data31=data3.copy()
data31['교통인프라지수_norm'] = 0

# 적용
for i in range(len(norm_weights)):
    data31['교통인프라지수_norm'] += norm_weights[i] * X.iloc[:, i]


sc=data31.sort_values(by='교통인프라지수_norm', ascending=False)

다중공산성을 보았을때, 모든변수의 VIF <=10이하임

In [12]:
result=sc[['시군구코드','교통인프라지수_norm']]
result.to_csv('/content/교통인프라지수_최종(3).csv')